In [11]:
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop --quiet

# Download the data folder at https://github.com/goodchemistryco/Tangelo-Examples/tree/main/examples/chemistry/data
import os
if not os.path.isdir("data"):
    !sudo apt install subversion
    !svn checkout https://github.com/goodchemistryco/Tangelo-Examples/branches/main/examples/chemistry/data

In [30]:
from tangelo import SecondQuantizedMolecule as SQMol
Cr=  """Cr 0.  0. 0.
                    """

# 2 electrons in 2 orbitals
fo = [0]
# Runs RHF calculation
mol_Cr = SQMol(Cr, q=6, spin=0, basis='3-21G', frozen_orbitals=None, symmetry=True)

# Runs ROHF calculation
mol_Cr = SQMol(Cr, q=6, spin=2, basis="3-21G", frozen_orbitals=None, symmetry=True)

In [31]:
# Symmetry labels and occupations for frozen core and active orbitals
print("  #  Energy  Symm Occ")
for i in range(4):
    print(f"{i+1:3d}{mol_Cr.mo_energies[i]: 9.4f}  {mol_Cr.mo_symm_labels[i]}   {int(mol_Cr.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_Cr.n_active_electrons}")
print(f"Number of active orbtials: {mol_Cr.n_active_mos}")

  #  Energy  Symm Occ
  1-224.0439  s+0   2
  2 -31.0336  s+0   2
  3 -26.9576  p-1   2
  4 -26.9576  p+1   2
Number of active electrons: 18
Number of active orbtials: 27


In [33]:
!pip install --prefer-binary pyscf==2.3.0

  Using cached pyscf-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.1 kB)
Using cached pyscf-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (47.2 MB)
  Attempting uninstall: pyscf
    Found existing installation: pyscf 2.6.2
    Uninstalling pyscf-2.6.2:
      Successfully uninstalled pyscf-2.6.2


In [40]:
from pyscf import mcscf

myhf = mol_Cr.mean_field
ncore = {"s+0": 1, "p-1": 1}
ncas = {"s+0": 1, "p+1": 1}

print("Calculation for s+0 symmetry")
mc = mcscf.CASCI(myhf, 2, (1, 1))
mo = mc.sort_mo_by_irrep(cas_irrep_nocc=ncas, cas_irrep_ncore=ncore)
mc.fcisolver.wfnsym = "s+0"
mc.fcisolver.nroots = 2
emc_s0 = mc.casci(mo)[0]



Calculation for s+0 symmetry

WARN: Mulitple states found in CASCI solver. First state is used to compute the Fock matrix and natural orbitals in active space.

CASCI state   0  E = -1028.47375478077  E(CI) = -65.2866539406350  S^2 = 0.0000000
CASCI state   1  E = -967.382859495021  E(CI) = -4.19575865488662  S^2 = 0.0000000


In [ ]:
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
from tangelo.algorithms.classical import FCISolver

# Dictionary of resources for each algorithm
algorithm_resources = dict()

# Ground state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_Cr, "ansatz": BuiltInAnsatze.UCCSD}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")
print(f"CASCI energy = {FCISolver(mol_Cr).simulate()}")
algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()

# First excited state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_Cr_t, "ansatz": BuiltInAnsatze.UpCCGSD}
vqe_solver_t = VQESolver(vqe_options)
vqe_solver_t.build()
vqe_energy_t = vqe_solver_t.simulate()
print("\n Lowest Triplet state")
print(f"VQE energy = {vqe_energy_t}")
print(f"CASCI energy = {FCISolver(mol_Cr_t).simulate()}")
algorithm_resources["vqe_triplet_state"] = vqe_solver_t.get_resources()

In [16]:
# Add initial VQE optimal circuit to the deflation circuits list
deflation_circuits = [vqe_solver.optimal_circuit.copy()]

# Calculate first and second excited states by adding optimal circuits to deflation_circuits
for i in range(2):
    vqe_options = {"molecule": mol_Cr, "ansatz": BuiltInAnsatze.UpCCGSD, 
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 0.4}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver.get_resources()

    deflation_circuits.append(vqe_solver.optimal_circuit.copy())

Excited state #1 	 VQE energy = -14.784036828629606
Excited state #2 	 VQE energy = -14.68019633282607


In [17]:
vqe_options = {"molecule": mol_Cr, "ansatz": BuiltInAnsatze.UpCCGSD, 
               "deflation_circuits": deflation_circuits,
               "deflation_coeff": 0.4, "ref_state": [1, 0, 1, 0]}
vqe_solver_triplet = VQESolver(vqe_options)
vqe_solver_triplet.build()
vqe_energy = vqe_solver_triplet.simulate()
print(f"VQE energy = {vqe_energy}")
algorithm_resources[f"vqe_deflation_state_{3}"] = vqe_solver_triplet.get_resources()

VQE energy = -14.838766460607102
